<a href="https://colab.research.google.com/github/Kanoru01/eagles-final-project/blob/master/Amazon_Fashion_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CALL VOLUME PREDICTION

## Business Understanding

### Problem statement
As a result of an increased number of reported cases on child abuse, there is a dire need to have prompt responses. It is crucial that reported cases are acted upon promptly to ensure timely and efficient handling of reported cases.

The stakeholders are Mtoto News, a digital company that leverages on technology to improve child wellbeing, and Childline Kenya, an NGO established in response to the state of child protection in Kenya and the manner in which abuse cases were being reported and handled.

### Main objective
Building a forecast model that predicts the number of incoming calls that Childline Kenya will receive per hour per day.

### Business Questions
Find out whether time affects child abuse occurences.<br>
Does weather/climate affect child abuse occurunces?<br>
Do public holidays affect child abuse occurences?<br>
Does child abuse occur more often at school or at home?<br>




## DATA UNDERSTANDING

The dataset is provided on [Zindi Africa](https://zindi.africa/competitions/mtoto-news-childline-kenya-call-volume-prediction-challenge), by Mtoto News and Childline Kenya. The data has been split into a test and training set. The training set contains all the calls (over 135,000) that were received from 1 January 2016 to 12 July 2016.

Each call contains the following fields:

- calldate - Date (month-day-year) and time of the call

- cc_status - Case status

- maincat - Main category call falls into

- subcat1 - Subcategory call falls under

- casepriority - Priority of the case

- referal -  Party contacted (refered to)

- caller_gender - Gender of the caller

- caller_age - Age of the caller

- caller_county - Area where the call came from

- child_age - Age of the child in case

- child_gender - Gender of the child in case

- child_county - Area where the child is from

- parent_age - Age of the parent

- parent_gender - Gender of the parent

- parent_county - Area where the child is from

- Abuser_Relationship - Relationship abuser has with the child in case

- Neglector_Relationship - Relationship neglector has with the child in case

- Physical_abuser_Relationship - Relationship physical abuser has with the child in case